In [115]:
from bs4 import BeautifulSoup
import requests
import re

# Class Paper and search functions

In [116]:
# Class Paper

class Paper(object):
    """
    Class for arxiv papers.
    """
    def __init__(self, identifier, title, author_list, abstract, replacement = False, print_abstract = False, max_authors = 3):
        self.identifier = identifier
        self.title = title
        self.author_list = author_list
        self.abstract = abstract
        self.replacement = replacement
        self.print_abstract = print_abstract
        self.max_authors = max_authors
        
    def GetIdentifier(self):
        return self.identifier
    
    def GetTitle(self):
        return self.title
    
    def GetAuthors(self):
        return selft.author_list
    
    def GetAbstract(self):
        return self.abstract
    
    def GetUrl(self):
        return "https://arxiv.org/abs/"+str(self.identifier)
    
    def IsReplacement(self):
        return self.replacement
    
    def __str__(self):
        paper_str = str(self.title) + "\n"
        ind_max = min(len(self.author_list), self.max_authors)
        for ind in range(ind_max):
            paper_str += self.author_list[ind]
            if ind < (ind_max - 1):
                paper_str += ", "
        if len(self.author_list)> self.max_authors:
            paper_str += " et al."
        paper_str += "\n"+self.GetUrl()
        if self.print_abstract: 
            paper_str += "\n" + self.abstract[:500]
        return paper_str

In [128]:
# Search functions

def SearchNewPapers(urls, Keywords_vect, print_abstract = False, print_replacement = False, repetition = False):
    """
    Search the list of keywords contained in Keywords_vect in titles,
    authors' lists, and abstracts of all papers from a list urls
    """
    
    all_interesting_papers_ids = set()
    
    for url in urls:
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html5lib')
        
        # Get the date
        announcement_string = soup.find("div", {"class": "list-dateline"}).text.split("announced ")

        # Building the lists of paper identifiers
        all_dt = soup('dt') # numbers and identifiers are contained in dt sections

        all_identifiers = [a for dt in all_dt for a in dt('a') if a.has_attr('href')]
        all_identifiers_list_tmp = [identifier.text.split()[0] for identifier in all_identifiers]

        regex=r"^arXiv"
        all_identifiers_list = [identifier.replace("arXiv:","") for identifier in all_identifiers_list_tmp if re.match(regex,identifier)]

        # Building the list of titles, authors, and abstracts
        all_dd = soup('dd') # titles, authors, and abstracts are contained in dd sections

        all_titles = [div for dd in all_dd for div in dd('div','list-title mathjax')]
        all_authors = [div for dd in all_dd for div in dd('div','list-authors')]
        all_abstracts = [p for dd in all_dd for p in dd('p','mathjax')]
        
        all_titles_list = [title.text.replace("\n","").replace("Title: ","").replace("  "," ") for title in all_titles]
        all_authors_list = [authors.text.replace("\n","").replace("Authors: ","").replace("  "," ").split(", ") for authors in all_authors]
        all_abstracts_list = [abstract.text.replace("\n","").replace("  "," ") for abstract in all_abstracts]

        all_titles_list_lower = [title.text.lower().replace("\n","").replace("title: ","").replace("  "," ") for title in all_titles]
        all_authors_list_lower = [authors.text.lower().replace("\n","").replace("authors: ","").replace("  "," ").split(", ") for authors in all_authors]
        all_abstracts_list_lower = [abstract.text.lower().replace("\n","").replace("  "," ") for abstract in all_abstracts]

        # Search for keywords in titles, authors' lists, and abstracts

        interesting_papers = set()

        for keyword in Keywords_vect:
            interesting_titles = set([index for index, title in enumerate(all_titles_list_lower) if keyword in title])
            interesting_authors = set([index for index, author in enumerate(all_authors_list_lower) if keyword in author])
            interesting_abstract = set([index for index, abstract in enumerate(all_abstracts_list_lower) if keyword in abstract])
            
            interesting_papers = interesting_papers.union(interesting_titles, interesting_authors,interesting_abstract)
            
        interesting_papers_list = sorted(interesting_papers)

        # Building the output with link to the papers

        # Taking care of replacements
        new_papers = len(all_abstracts_list)
        total_papers = len(all_identifiers_list)

        all_abstracts_list_full = all_abstracts_list
        for item in range(new_papers,total_papers):
            all_abstracts_list_full.append("This paper is a replacement.")

        # Printing results

        print("Today ({})".format(announcement_string[1]),"in", url,"there are", new_papers, "new papers and", total_papers-new_papers, "replacements.\n\n")

        todays_papers = []
        for item in interesting_papers_list:
            if item <= new_papers:
                replacement = False
            else: 
                replacement = True
            paper = Paper(all_identifiers_list[item], all_titles_list[item], all_authors_list[item], all_abstracts_list[item], replacement, print_abstract)
            todays_papers.append(paper)

        new_interesting_papers = 0
        already_announced_papers = 0
        for paper in todays_papers:
            if not paper.IsReplacement():
                if paper.GetIdentifier() not in all_interesting_papers_ids:
                    new_interesting_papers += 1
                else:
                    already_announced_papers += 1

        print("There are",new_interesting_papers,"new interesting papers and", len(todays_papers)-new_interesting_papers-already_announced_papers,"replacements. \n")
        
        ind=0
        for paper in todays_papers:
            if not print_replacement:
                if not paper.IsReplacement():
                    if paper.GetIdentifier() not in all_interesting_papers_ids:
                        print(str(ind+1)+")")
                        print(paper,"\n")
                        ind += 1
            else: 
                if paper.GetIdentifier() not in all_interesting_papers_ids:
                    print(str(ind+1)+")")
                    print(paper,"\n")
                    ind += 1
                
        # Creating a set with all ids of already announced papers to avoid repetition
        if not repetition:
            for paper in todays_papers: 
                all_interesting_papers_ids.add(paper.GetIdentifier())

        print("\n************************************\n")

# Main program

In [129]:
#Import keywords from txt file
#Format: Insert keywords as new lines. 
#Lines beginning with # will be ignored. 
#Check if there are accidental white space at the end of each entry!

Keywords_vect=[]

with open('Keywords.txt') as file_for_reading:
    for line in file_for_reading:
        if not re.match('^#', line) and line.strip()!='':
            Keywords_vect.append(line.strip().lower())

#print(Keywords_vect)

In [130]:
urls  = ["https://arxiv.org/list/cond-mat/new",
         "https://arxiv.org/list/quant-ph/new",
         "https://arxiv.org/list/physics.atom-ph/new"]

print_abstract = True
print_replacement = False
repetition = False               # if True, paper already announced in previous section will be announced again
SearchNewPapers(urls, Keywords_vect, print_abstract, print_replacement, repetition)

Today (Mon, 15 Jun 20) in https://arxiv.org/list/cond-mat/new there are 66 new papers and 42 replacements.


There are 16 new interesting papers and 4 replacements. 

1)
Low-frequency divergence and quantum geometry of the bulk photovoltaic effect in topological semimetals
Junyeong Ahn, Naoto Nagaosa
https://arxiv.org/abs/2006.06709
We study the low-frequency properties of the bulk photovoltaic effect intopological semimetals. The bulk photovoltaic effect is a nonlinear opticaleffect that generates DC photocurrents under uniform irradiation, allowed bynoncentrosymmetry. It is a promising mechanism for a terahertz photodetectionbased on topological semimetals. Here, we systematically investigate thelow-frequency behavior of the second-order optical conductivity in point-nodesemimetals. Through symmetry and power-counting ana 

2)
Light-wave Control of Non-equilibrium Correlated States using Quantum Femtosecond Magnetism and Time-Periodic Modulation of Coherent Transport
P. C. Lingos, M.

Today (Mon, 15 Jun 20) in https://arxiv.org/list/quant-ph/new there are 26 new papers and 19 replacements.


There are 4 new interesting papers and 3 replacements. 

1)
Mechanical oscillator thermometry in the nonlinear optomechanical regime
Victor Montenegro, Marco G. Genoni, Abolfazl Bayat et al.
https://arxiv.org/abs/2006.06699
Optomechanical systems are promising platforms for controlled light-matterinteractions. They are capable of providing several fundamental and practicalnovel features when the mechanical oscillator is cooled down to nearly reachits ground state. In this framework, measuring the effective temperature of theoscillator is perhaps the most relevant step in the characterization of thosesystems. In conventional schemes, the cavity is driven strongly, and theoverall system is well-described by a linear  

2)
Self-testing mutually unbiased bases in higher dimensions with space-division multiplexing optical fiber technology
Máté Farkas, Nayda Guerrero, Jaime Cariñe et 